In [1]:
from decimal import Decimal
import pandas as pd
import datetime
import numpy as np
import os
from pyexcelerate import Workbook
pd.set_option('display.max_columns', None)

# впр    
def insert_vpr(x,dictt,name):
    try:
        return dictt[x][name]
    except KeyError:
        return 0
    
def rev_to_dict(ind,find):
#     ind = ind.astype('str')
    df = pd.DataFrame(index=ind,data=find.values,columns=[find.name])
    df = df.groupby(df.index).first()
    dictt =  df.to_dict('index')
    return dictt  


def func1(x):
    if x['Признак ЦС']=='4':
        return x['округ']+x['Завод']
    elif  x['Признак ЦС']=='3':#     округ+склад
        return x['Область ППМ']
    elif  x['Признак ЦС']=='1' and x['Партия']in ['DAMAGED', 'REPAIR'] :
        return 'R'
    elif  x['Признак ЦС']=='1' and x['Партия'].find('FREE')!=-1  :
        return x['округ']
    elif x['Признак ЦС']=='1' and len(x['Партия'])==2 and x['Партия']==x['округ']:
        return x['Партия']
    elif x['Признак ЦС']=='1' and len(x['Партия'])==6 and x['Партия'][:2].isalpha():
        return x['Партия']
    elif x['Признак ЦС']=='1' and len(x['Партия'])==10 and x['Партия'].isalpha()==False:
        return x['округ']
    else:
        return ''
    
#     elif  x['Признак ЦС']=='1' and (x['Партия']!='DAMAGED' or x['Партия']!='REPAIR' or x['Партия'] in kod_okr ):# Партия МВП
#         return x['округ']
#     else:
#         return x['Партия']
    
def get_in_backup(path_backup,name):
    try:
        backup = pd.HDFStore(path_backup)
        ar  = backup[name]
        backup.close()
        return ar
    except:
        backup.close()    
        
# превращает цифры строками  в нормальные цифры
def replace_float(df,name_cols):
    try:
        for col_n in name_cols:
            mas = df[col_n].apply(lambda x: x.replace('.',''))
            mas1 = mas.apply(lambda x: x.replace(',','.'))
            df[col_n] =  mas1.astype('float')
    except:
        pass      
    
    
def modul(x):
    if len(list(str(x['Заказ'])))==0:
        x['Заказ'] = 0
    if  x['Вид документа потребности'] in ['UB02']:
        return 'прочая потребность'
    elif x['Вид документа потребности']=='AR' and list(str(x['Заказ']))[0]=='1':
        return 'ТОРО'
    elif ((x['Вид документа потребности']=='AR' and list(str(x['Заказ']))[0] in['2','4'])) or x['Вид документа потребности']=='SB':
        return 'производство'
    elif  x['Вид документа потребности'] in ['Z1','Z3']:
        return 'сбыт'
    else:
        return ''
    
    
def modul1(x):
    if len(list(str(x['Заказ'])))==0:
        x['Заказ'] = 0
        
    if x['Вид движения']  in['261','262'] and list(str(x['Заказ']))[0]=='1':
        return 'ТОРО'
    elif x['Вид движения']  in['261','262'] and list(str(x['Заказ']))[0]!='1':
        return 'производство'
    elif  x['Вид движения'] in ['Z03','Z04','Z15','Z16','Z19','Z20']:
        return 'прочая потребность' 
    else:
        return ''
    
    
def save_pyex(path,df):
    wb = Workbook()
    wb.new_sheet('sheet1',data =  [df.columns.tolist(),]+df.values.tolist())
    wb.save(path)    
    
def func_KONH(x):
    if x['Таблица']=='900':return x['Переменный ключ']
    elif x['Таблица']=='901':return x['Переменный ключ'][4:]
    elif x['Таблица']=='902':return x['Переменный ключ'][3:]
    else:return 'пусто'    
    
    
import numpy as np
import time
from multiprocessing import Pool
import functions
def parallel_calc(cons,KONH,current_date,KONP,MARM):
    start_time = time.time()
    pool = Pool(processes=6)
    arrs =  np.array_split(cons,6)
    arrs =  [(arr,KONH,current_date,KONP,MARM) for arr in arrs]
    df_list = pool.map(functions.calc_price, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    return df   


import numpy as np
import time
from multiprocessing import Pool
import functions
def parallel_calc_fast(cons,KONH,current_date,KONP,MARM):
    start_time = time.time()
    pool = Pool(processes=6)
    KONH_po = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. по'].apply(lambda x:str(x.year)))])
    KONH_po.rename(columns={"Номер записи условий": 'Номер_записи_условий'},inplace=True)
    KONH_s = KONH.reset_index().set_index(['Материал',pd.Index(KONH['Действ. с'].apply(lambda x:str(x.year)))])
    KONH_s.rename(columns={"Номер записи условий": 'Номер_записи_условий'},inplace=True)
    KONP = KONP.set_index(['Номер записи условий'])
    MARM =  MARM.set_index(['Материал','Альтернативная ЕИ'])
    arrs =  np.array_split(cons,6)
    arrs =  [(arr,KONH_po,KONH_s,current_date,KONP,MARM) for arr in arrs]
    df_list = pool.map(functions.calc_price_super_fast, arrs)
    df = pd.concat(df_list)
    print("--- %s seconds ---" % (time.time() - start_time))
    pool.close()
    return df 


def save_backup(path_backup,arrs):
    try:
        backup = pd.HDFStore(path_backup)
        for arr in arrs:
            backup[str(arr.name)] = arr
        backup.close()   
    except:
        backup.close()
    
def get_in_backup(path_backup,name):
    try:
        backup = pd.HDFStore(path_backup)
        ar  = backup[name]
        backup.close()
        return ar
    except:
        backup.close()
        
def sort_for_modul(x):
    if x['Модуль']=='ТОРО':
        return 1
    elif  x['Модуль']=='производство':
        return 2
    elif  x['Модуль']=='сбыт':
        return 3
    elif  x['Модуль']=='прочая потребность':
        return 4  
    else:
        return ''
        
def part_MVP(x):
    if x['Признак ЦС']=='1':
        return x['округ']
    elif  x['Признак ЦС']=='4':
        return x['округ']+x['Завод']
    elif  x['Признак ЦС']=='3':
        return x['Область ППМ']   
    else:
        return ''
    
def part_na_zav_CS(x):
    if x['Материал'].find('R')!=-1:
        return 'R'
    elif x['Признак ЦС']=='1':
        return x['округ']
    elif  x['Признак ЦС']=='4':
        return x['округ']+x['Завод']
    elif  x['Признак ЦС']=='3':
        return str(x['округ'])+str(x['БЕ'])[:1]+str(x['Склад поступления'])[-3:]    
    else:
        return ''
    
def pr_ppm(x,spr_pr_uchstiya_v_ppm):
    if x['Вид движения'] in['101','102']:
        return insert_vpr(x['Завод']+x['Склад'],spr_pr_uchstiya_v_ppm,'Код ППМ')
    if x['Вид движения'] in['641','642']:
        return insert_vpr(x['Завод']+x['Принимающий склад'],spr_pr_uchstiya_v_ppm,'Код ППМ')  
    else:
        return ''
    
    
def proc_otklon(x):
    if x['Количество потребности']==0:
        return 1
    elif x['Отклонение ФС-ПП кол-во']>=0:
        return 1
    else:
        return x['Кол-во в ЕИ ввода']/x['Количество потребности']   

def proc_otklon_d(x):
    if x['Сумма потребности']==0:
        return 1
    elif x['Отклонение ФС-ПП кол-во_Д']>=0:
        return 1
    else:
        return (x['Сумма во ВВ']/345)/x['Сумма потребности']      
    
# def primechaniya(x):
#     if x['Кол-во в ЕИ ввода']>0 and x['Количество потребности']==0:
#         return 'списано без потребности на начало месяца'
#     elif x['Отклонение ФС-ПП кол-во']>0:
#         return 'списано больше потребности на начало месяца'
#     elif x['Отклонение ФС-ПП кол-во']==0:
#         return 'списано без отклонений' 
#     elif (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']>=1 or x['Обеспеченность ПП запасом на начало периода %']>=1) and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']>0: 
#         return 'списано частично, потребность обеспечена в полном объеме'
#     elif (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']>=1 or x['Обеспеченность ПП запасом на начало периода %']>=1) and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']==0: 
#         return 'не списано, потребность обеспечена в полном объеме'
#     elif x['Обеспеченность ПП запасом на начало периода и ФП %']>=1 and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']>0:
#         return 'списано частично, потребность обеспечена с запозданием'
#     elif x['Обеспеченность ПП запасом на начало периода и ФП %']>=1 and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']==0:
#         return 'не списано, потребность обеспечена с запозданием'
#     elif x['Количество потребности']>0 and x['Отклонение ФС-ПП %']==0 and x['Обеспеченность ПП запасом на начало периода %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==0 and x['Кол-во в ЕИ ввода']==0: 
#         return 'не списано, потребность не обеспечена'
#     elif x['Количество потребности']>0 and x['Отклонение ФС-ПП %']==0 and x['Обеспеченность ПП запасом на начало периода %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==0 and x['Кол-во в ЕИ ввода']>0: 
#         return 'списано частично, потребность не обеспечена'
#     elif x['Количество потребности']>0 and x['Кол-во в ЕИ ввода']>0:
#         return 'списано частично, потребность обеспечена не в полном объеме'
#     elif x['Количество потребности']>0 and x['Кол-во в ЕИ ввода']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']<1:
#         return 'не списано, потребность обеспечена не в полном объеме'
#     else:
#         return 0
def primechaniya_po_spis(x):
    if x['Кол-во в ЕИ ввода']>0 and x['Количество потребности']==0:
        return 'списано без потребности'
    elif x['Отклонение ФС-ПП кол-во']>0:
        return 'списано больше потребности'
    elif x['Отклонение ФС-ПП кол-во']==0:
        return 'списано без отклонений' 
    elif (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']>=1 or x['Обеспеченность ПП запасом на начало периода %']>=1) and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']>0: 
        return 'списано частично'
    elif (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']>=1 or x['Обеспеченность ПП запасом на начало периода %']>=1) and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']==0: 
        return 'не списано'
    elif x['Обеспеченность ПП запасом на начало периода и ФП %']>=1 and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']>0:
        return 'списано частично'
    elif x['Обеспеченность ПП запасом на начало периода и ФП %']>=1 and x['Отклонение ФС-ПП кол-во']<0 and x['Кол-во в ЕИ ввода']==0:
        return 'не списано'
    elif x['Количество потребности']>0 and x['Отклонение ФС-ПП %']==0 and x['Обеспеченность ПП запасом на начало периода %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==0 and x['Кол-во в ЕИ ввода']==0: 
        return 'не списано'
    elif x['Количество потребности']>0 and x['Отклонение ФС-ПП %']==0 and x['Обеспеченность ПП запасом на начало периода %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==0 and x['Кол-во в ЕИ ввода']>0: 
        return 'списано частично'
    elif x['Количество потребности']>0 and x['Кол-во в ЕИ ввода']>0:
        return 'списано частично'
    elif x['Количество потребности']>0 and x['Кол-во в ЕИ ввода']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']<1:
        return 'не списано'
    else:
        return 0

def primechaniya_po_spis_d(x):
    if (x['Сумма во ВВ']/345)>0 and x['Сумма потребности']==0:
        return 'списано без потребности'
    elif x['Отклонение ФС-ПП кол-во_Д']>0:
        return 'списано больше потребности'
    elif x['Отклонение ФС-ПП кол-во_Д']==0:
        return 'списано без отклонений' 
    elif (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']>=1 or x['Обеспеченность ПП запасом на начало периода %_Д']>=1) and x['Отклонение ФС-ПП кол-во_Д']<0 and (x['Сумма во ВВ']/345)>0: 
        return 'списано частично'
    elif (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']>=1 or x['Обеспеченность ПП запасом на начало периода %_Д']>=1) and x['Отклонение ФС-ПП кол-во_Д']<0 and (x['Сумма во ВВ']/345)==0: 
        return 'не списано'
    elif x['Обеспеченность ПП запасом на начало периода и ФП %_Д']>=1 and x['Отклонение ФС-ПП кол-во_Д']<0 and (x['Сумма во ВВ']/345)>0:
        return 'списано частично'
    elif x['Обеспеченность ПП запасом на начало периода и ФП %_Д']>=1 and x['Отклонение ФС-ПП кол-во_Д']<0 and (x['Сумма во ВВ']/345)==0:
        return 'не списано'
    elif x['Сумма потребности']>0 and x['Отклонение ФС-ПП %_Д']==0 and x['Обеспеченность ПП запасом на начало периода %_Д']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %_Д']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']==0 and (x['Сумма во ВВ']/345)==0: 
        return 'не списано'
    elif x['Сумма потребности']>0 and x['Отклонение ФС-ПП %_Д']==0 and x['Обеспеченность ПП запасом на начало периода %_Д']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %_Д']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']==0 and (x['Сумма во ВВ']/345)>0: 
        return 'списано частично'
    elif x['Сумма потребности']>0 and (x['Сумма во ВВ']/345)>0:
        return 'списано частично'
    elif x['Сумма потребности']>0 and (x['Сумма во ВВ']/345)==0 and x['Обеспеченность ПП запасом на начало периода и ФП %_Д']<1:
        return 'не списано'
    else:
        return 0
    
def primechaniya_po_obespech(x):
    if x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==1:
        return 'Потребность обеспечена в полном объеме'
    elif  x['Обеспеченность ПП запасом на начало периода и ФП %']==1:
        return 'Потребность обеспечена с запозданием'
    
    elif x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %']==0 and x['Обеспеченность ПП запасом на начало периода %']==0:
        return 'Потребность не обеспечена'
    else:
        return 'Потребность обеспечена частично'

    
def primechaniya_po_obespech_d(x):
    if x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']==1 or x['Отклонение ФС-ПП кол-во_Д']>=0:
        return 'Потребность обеспечена в полном объеме'
    elif  x['Обеспеченность ПП запасом на начало периода и ФП %_Д']==1:
        return 'Потребность обеспечена с запозданием'
    
    elif x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']==0 and x['Обеспеченность ПП запасом на начало периода и ФП %_Д']==0 and (x['Сумма во ВВ']/345)==0 and (x['Сумма Распределение_остатака  на начало']/345)==0:
        return 'Потребность не обеспечена'
    else:
        return 'Потребность обеспечена частично'


    
def KPI_prichinam(x):
    if x['KPI по вовлечению']<Decimal('0.8'):
        return 'красная'
    elif x['KPI по вовлечению']>=Decimal('0.95'):
        return 'зеленая'
    else:
        return 'желтая'

def KPI_prichinam_d(x):
    if x['KPI по вовлечению_Д']<Decimal('0.8'):
        return 'красная'
    elif x['KPI по вовлечению_Д']>=Decimal('0.95'):
        return 'зеленая'
    else:
        return 'желтая'
    
def KPI_vovlech(x):
    if x['Отклонение ФС-ПП %']<1 and  x['Отклонение ФС-ПП %']>= x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %'] and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']!=0:
        return 1
    else:
        return x['Отклонение ФС-ПП %']
def KPI_vovlech_d(x):
    if x['Отклонение ФС-ПП %_Д']<1 and  x['Отклонение ФС-ПП %_Д']>= x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д'] and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %_Д']!=0:
        return 1
    else:
        return x['Отклонение ФС-ПП %_Д']
    
def dunc_tcurr(d):
    if len([x for x in d['Курс'] if x=='.'])==2:
        a = list(d['Курс'])
        a[d['Курс'].find('.')] = ''
        st = ''.join(a)
        return st
    else:
        return d['Курс']
def otklon_zak_sum_USD(x):
#     if x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']<1 and (x['Распределение_остатака  на начало']+x['1 декада']+x['2 декада']+x['3 декада']+x['Погрешность прихода'])==0:
#         return (x['Количество потребности']-x['Кол-во в ЕИ ввода'])*x['Плановая цена']
#     elif x['3 декада']!= 0:
#         return x['3 декада']*x['Плановая цена']
#     else:
#         return 0
    return x['Условное отклонение ФС-ПП, сумма USD']-x['Отклонение  заказчика, сумма USD']

def pokaz_vovl(x):
    if x['Кол-во в ЕИ ввода']==0 and x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) кол-во']==0:
        return ''
    elif (x['Кол-во в ЕИ ввода']- x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) кол-во'])>=0:
        return 1
    else:
        return x['Кол-во в ЕИ ввода']/ x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) кол-во']
    

def header_parser(df):

    lst = list(df.columns)

    value = 0
    for index,col in enumerate(lst):
        if col is np.nan:
            lst[index] = lst[index-1]+str(value)
            value +=1
        else:
            value  = 0

    a,b = [],[]
    for col in lst:
        a.append(col)
        b.append(lst.count(col))

    a = pd.Series(a)
    b = pd.Series(b)
    b.name = 'val'    
    s  = rev_to_dict(a,b)

    ls_k = list(s.keys())

    for col in ls_k:
        if s[col]['val']>1:
            for ind in range(0,s[col]['val']):
                for ind_,col_ in enumerate(lst):
                    if col_==col:
                        lst[ind_] = col_+str(ind)
                        break

    df.columns = lst
    
    return df
    

In [2]:
# управление механикой остатков(вызывать ее)
def ostat(ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    
       
    pz_ost = ost_st
    cons = potr_st.sort_values(['Область ППМ','сортировка по модулю'])
    cons = cons.set_index(['Материал','Область ППМ','партия МВП'])
    
    # 15,2,1
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try:
            df_master = cons.xs([pz_ost.loc[pz_slave_row,'Материал'],
                         pz_ost.loc[pz_slave_row,'Область ППМ'],
                        pz_ost.loc[pz_slave_row,'Партия']])    
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            df_master = df_master.sort_values(['Материал','Область ППМ','партия МВП'])

            mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
#             print(len(df_master))
#             cons = cons.reset_index().set_index(['Материал','Область ППМ','партия МВП'])

                
                
                
                 
            #             распределяем
             
        
    
    print('15.2.1 закончили')
    
    cons = cons.reset_index().set_index(['Материал','Завод пополнения','партия на заводе ЦС'])
    
    # 15,2,2
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        try:
            df_master = cons.xs([pz_ost.loc[pz_slave_row,'Материал'],
                         pz_ost.loc[pz_slave_row,'Область ППМ'],
                        pz_ost.loc[pz_slave_row,'Партия']])    
        except KeyError:
            df_master = pd.DataFrame()
            
        if len(df_master)>0:
            df_master = df_master.sort_values(['Материал','Завод пополнения','партия на заводе ЦС'])

            mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
#             print(len(df_master))
#             cons = cons.reset_index().set_index(['Материал','Завод пополнения','партия на заводе ЦС'])
                
                
                 
            #             распределяем
             

        
    print('15.2.2 закончили')
    
    cons = cons.reset_index().set_index(['Материал','Завод пополнения','округ'])
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
        try:
            df_master = cons.xs([pz_ost.loc[pz_slave_row,'Материал'],
                         pz_ost.loc[pz_slave_row,'Область ППМ'],
                        pz_ost.loc[pz_slave_row,'Партия']])    
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            df_master = df_master.sort_values(['Материал','Завод пополнения','округ'])

            mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5)
#             print(len(df_master))
#             cons = cons.reset_index().set_index(['Материал','Завод пополнения','округ'])
                
                
                 
            #             распределяем
             
    print('15.2.3 закончили')
    

# мехника распределения остатков 
# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой потребности
# col_name_4 название столбца с начальным кол-вом  потребности
def mex_ost(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3,col_name_4,col_name_5):
    df_master = df_master.reset_index().set_index(['index'])
#     print(df_master.index)
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
        if len_it == ind:
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            summa = ostatok*price
            
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

            pz_ost.at[pz_slave_row, col_name_0] = 0
            ost_st.at[pz_slave_row, col_name_0] = 0
            # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
            ostatok = 0
            return
        
        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:    
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
                
                summa = vzyali*price
                
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
                if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                    price = 0
                else:
                    price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
                summa = ostatok*price
                potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            if pz_ost.at[pz_slave_row, col_name_3]==0 or pz_ost.at[pz_slave_row, col_name_4]==0:
                price = 0
            else:
                price = pz_ost.at[pz_slave_row, col_name_3]/pz_ost.at[pz_slave_row, col_name_4]
            
            vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
            
            summa = vzyali*price
            potr_st.at[row_master_index, col_name_5] = potr_st.at[row_master_index, col_name_5] + summa
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [3]:


# управление механикой остатков(вызывать ее)
def prihod_raspr(ost_st, potr_st,col_name_0,col_name_1,col_name_2):
    
    pz_ost1 =   ost_st 
    pz_ost = pz_ost1[pz_ost1['Декада проводки']=='1 декада'].sort_values('Дата проводки')
    
    cons = potr_st.sort_values(['Область ППМ','сортировка по модулю','Материал','Завод пополнения','округ'])
    cons = cons.set_index(['Материал','Завод пополнения','округ'])

    # 15,2,1
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try:
            df_master = cons.xs([pz_ost.loc[pz_slave_row,'Материал'],
                         pz_ost.loc[pz_slave_row,'Область ППМ'],
                        pz_ost.loc[pz_slave_row,'округ']])    
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            df_master = df_master.sort_values(['Материал','Завод пополнения','округ'])
#             print(df_master)
            mex_prihod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,'1 декада')
#             print(len(df_master))
#             cons = cons.reset_index().set_index(['Материал','Завод пополнения','округ'])
    print('15.2.1 закончили')
    
    
    

                
                 
            #             распределяем
             
    cons = potr_st.sort_values(['Область ППМ','сортировка по модулю','Материал','Завод пополнения','округ'])
    cons = cons.set_index(['Материал','Завод пополнения','округ'])
    pz_ost = pz_ost1[pz_ost1['Декада проводки']=='2 декада'].sort_values('Дата проводки')
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try:
            df_master = cons.xs([pz_ost.loc[pz_slave_row,'Материал'],
                         pz_ost.loc[pz_slave_row,'Область ППМ'],
                        pz_ost.loc[pz_slave_row,'округ']])    
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            df_master = df_master.sort_values(['Материал','Завод пополнения','округ'])
            mex_prihod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,'2 декада')
#             print(len(df_master))
#             cons = cons.reset_index().set_index(['Материал','Завод пополнения','округ'])
    print('15.2.2 закончили')
                
                 
            #             распределяем
             
    cons = potr_st.sort_values(['Область ППМ','сортировка по модулю','Материал','Завод пополнения','округ'])
    cons = cons.set_index(['Материал','Завод пополнения','округ'])
    pz_ost = pz_ost1[pz_ost1['Декада проводки']=='3 декада'].sort_values('Дата проводки')    
    for pz_slave_row in pz_ost[pz_ost[col_name_0]!=0].index:
#         print(pz_ost.loc[pz_slave_row,'Материал'])
        try:
            df_master = cons.xs([pz_ost.loc[pz_slave_row,'Материал'],
                         pz_ost.loc[pz_slave_row,'Область ППМ'],
                        pz_ost.loc[pz_slave_row,'округ']])    
        except KeyError:
            df_master = pd.DataFrame()

        if len(df_master)>0:
            df_master = df_master.sort_values(['Материал','Завод пополнения','округ'])
            
            mex_prihod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,'3 декада')
#             print(len(df_master))
#             cons = cons.reset_index().set_index(['Материал','Завод пополнения','округ'])
    print('15.2.3 закончили')
        
        
        




# мехника распределения остатков 
# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
def mex_prihod(df_master, pz_ost, pz_slave_row, ost_st, potr_st,col_name_0,col_name_1,col_name_2,col_name_3):
    df_master = df_master.reset_index().set_index(['index'])
    ostatok = pz_ost.at[pz_slave_row, col_name_0]
    len_it = len(df_master) - 1
    for ind, row_master_index in enumerate(df_master.index):
#         print(potr_st.at[row_master_index, col_name_2])
#         print(potr_st.at[row_master_index, col_name_1])
#         print(ostatok)
#         print('/////////')
#         if len_it == ind:
           
#             potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok

#             pz_ost.at[pz_slave_row, col_name_0] = 0
#             ost_st.at[pz_slave_row, col_name_0] = 0
#             # print(df_master[['Количество потребности','Дата потребности','Распределение_остатака']])
#             ostatok = 0
#             return
        
        if (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (
                potr_st.at[row_master_index, col_name_2] >= ostatok):
            if ostatok+potr_st.at[row_master_index, col_name_1]>potr_st.at[row_master_index, col_name_2]:
                a=0
                vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
    
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + vzyali
                potr_st.at[row_master_index, col_name_3] = potr_st.at[row_master_index, col_name_3] + vzyali

                pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0]-vzyali
                ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0]-vzyali
                ostatok =  ostatok -vzyali

            else:
               
                potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + ostatok
                potr_st.at[row_master_index, col_name_3] = potr_st.at[row_master_index, col_name_3] + ostatok
                pz_ost.at[pz_slave_row, col_name_0] = 0
                ost_st.at[pz_slave_row, col_name_0] = 0
                ostatok = 0
                # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])
                return

        elif (potr_st.at[row_master_index, col_name_2] > potr_st.at[row_master_index, col_name_1]) and (potr_st.at[row_master_index, col_name_2] < ostatok):
            vzyali = potr_st.at[row_master_index, col_name_2]-potr_st.at[row_master_index, col_name_1]
            potr_st.at[row_master_index, col_name_1] = potr_st.at[row_master_index, col_name_1] + (potr_st.at[row_master_index, col_name_2] -potr_st.at[row_master_index, col_name_1])
            potr_st.at[row_master_index, col_name_3] = potr_st.at[row_master_index, col_name_3] + (potr_st.at[row_master_index, col_name_2] -potr_st.loc[row_master_index, '1 декада':'3 декада'].sum())

            pz_ost.at[pz_slave_row, col_name_0] = pz_ost.at[pz_slave_row, col_name_0] - vzyali
            ost_st.at[pz_slave_row, col_name_0] = ost_st.at[pz_slave_row, col_name_0] - vzyali
            ostatok = ostatok - vzyali
            
            # print(df_master[['Количество потребности', 'Дата потребности', 'Распределение_остатака']])

In [4]:
# переменные

# directory = 'C:\\Users\\Илья\\Desktop\\kazzinc\\KPI\\'
# current_date = datetime.datetime(2019,12,1)
directory = 'C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\0421\\'
current_date = datetime.datetime(2021,4,1)


In [5]:
ost_kon_period =  pd.read_excel(directory+'Остатки на 30.04.2021.xlsx',sheet_name='Sheet1')
ost_nach_period =  pd.read_excel(directory+'Остатки на 31.03.2021.xlsx',sheet_name='Sheet1')
prihod = pd.read_excel(directory+'ФП.xlsx',sheet_name='Sheet1')
rashod = pd.read_excel(directory+'Расход за апрель.xlsx',sheet_name='Sheet1')


A900 = pd.read_excel(directory+'A900.xlsx',sheet_name='Sheet1')
A901 = pd.read_excel(directory+'A901.xlsx',sheet_name='Sheet1')
A902 = pd.read_excel(directory+'A902.xlsx',sheet_name='Sheet1')
block_zapas = pd.read_excel(directory+'Блокированный запас.xlsx',sheet_name='Sheet1')
spr_for_obl_ppm = pd.read_excel(directory+'MDLG.xlsx',sheet_name='Sheet1')
spr_kod_okruga_ = pd.read_excel(directory+'ZMM_E0078_MDLV.xlsx',sheet_name='Sheet1')
T001L = pd.read_excel(directory+'T001L.xlsx',sheet_name='Sheet1')
T001W = pd.read_excel(directory+'T001W.xlsx',sheet_name='Sheet1')
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})
ved_mat = pd.read_excel(directory+'Выгрузка по ведущему материалу.xlsx',sheet_name='Sheet1')
TCURR = pd.read_excel(directory+'TCURR.xlsx',sheet_name='Sheet1')
KONH = pd.read_excel(directory+'KONH.xlsx',sheet_name='Sheet1')
KONP  = pd.read_excel(directory+'KONP.xlsx',sheet_name='Sheet1')
PICRL = pd.read_excel(directory+'PICRL.xlsx',sheet_name='Sheet1')
PICPS = pd.read_excel(directory+'PICPS.xlsx',sheet_name='Sheet1')
MARM = pd.read_excel(directory+'MARM.xlsx',sheet_name='Sheet1')
MDLV = pd.read_excel(directory+'MDLV.xlsx',sheet_name='Sheet1')
T023 = pd.read_excel(directory+'T023.xlsx',sheet_name='Sheet1')
spr_kompleks = pd.read_excel(directory+'Завод склад комплекс.xlsx',sheet_name='Sheet1')
zap_col_st = pd.read_excel(directory+'Запасы количество и стоимость.xlsx',sheet_name='Лист1')
# import  os
# os.chdir('C:\\Users\\Илья\\')
# sprav_mat_cal_last_full = get_in_backup(path_backup ='backup.h5',name='sprav_mat_cal_last_full')
# os.chdir('C:\\Users\\Илья\\kpi')

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7,9,10,11,12,13,16,17,18,19,20,29,30,31,32,37,39,55,58,62,65,66,69,71,75,76,84,85,89,91,92,95,96,98,103,107) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# ost_nach_period.drop(['Кол-во запаса в начале периода','Стоимость в начале периода'],axis = 1,inplace = True)
# ost_kon_period.drop(['Кол-во запаса в начале периода','Стоимость в начале периода'],axis = 1,inplace = True)
ost_nach_period.rename(columns={"Кол-во запаса в конце периода": 'Кол-во запаса в начале периода','Ст-ть запаса в конце периода':'Стоимость в начале периода'},inplace=True)
block_zapas.rename(columns={"Номер материала": 'Материал','Краткий текст материала':'КрТекстМатериала'},inplace=True)
ost_cons = pd.concat([ost_nach_period,ost_kon_period])

ost_cons.fillna(0,inplace=True)
block_zapas.fillna(0,inplace=True)
T001L.fillna(0,inplace=True)
T001W.fillna(0,inplace=True)
ved_mat.fillna(0,inplace=True)
TCURR.fillna(0,inplace=True)
MDLV.fillna(0,inplace=True)
TCURR['Курс'] = TCURR['Курс'].apply(lambda x:x.replace('/',''))
TCURR['Действит. с'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in TCURR['Действит. с'].values]


TCURR['Курс'] = TCURR['Курс'].apply(lambda x: x.replace(',','.'))

TCURR['Курс'] = TCURR.apply(lambda d: dunc_tcurr(d),axis = 1)
TCURR['Курс'] = pd.to_numeric(TCURR['Курс'])

ost_cons = ost_cons[ost_cons['Материал']!=0]
block_zapas = block_zapas[block_zapas['Склад']!=0]




to_str_ost = ['Завод','Склад','Партия','Материал','БЕ','Особый запас','КрТекстМатериала','Базовая ЕИ']
to_str_blockzap = ['Материал','Завод','Склад','Партия','КрТекстМатериала','Базовая ЕИ']

for i in to_str_ost:
    ost_cons[i] = ost_cons[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    ost_cons[i] = ost_cons[i].apply(lambda x: '' if x=='0' else x)
for i in to_str_blockzap:
    block_zapas[i] = block_zapas[i].apply(lambda x: str(x).replace('.0','')).astype('str')    
    block_zapas[i] = block_zapas[i].apply(lambda x: '' if x=='0' else x)
    
    
to_str_MDLV = ['Область ППМ','Текст ОблПланирПотр','СкладПоступления']
for i in to_str_MDLV:
    MDLV[i] = MDLV[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    MDLV[i] = MDLV[i].apply(lambda x: '' if x=='0' else x)
    
to_str_T001L = ['Завод','Склад','Код ППМ']
for i in to_str_T001L:
    T001L[i] = T001L[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    T001L[i] = T001L[i].apply(lambda x: '' if x=='0' else x)    
    
ost_group = ost_cons.groupby(to_str_ost).agg({'Кол-во запаса в конце периода':'sum','Кол-во запаса в начале периода':'sum'\
                                 ,'Ст-ть запаса в конце периода':'sum','Стоимость в начале периода':'sum'}).reset_index()   



ostatok_with_block = pd.merge(ost_group,block_zapas.groupby(to_str_blockzap).agg({'Блокированный запас':'sum','Ст-ть/блок. запаса':'sum'}).reset_index(),how='outer')
ostatok_with_block.fillna(0,inplace=True)

spr_obl_ppm = rev_to_dict(spr_for_obl_ppm['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str')+spr_for_obl_ppm['Склад'].\
            apply(lambda x: str(x).replace('.0','')).astype('str'),spr_for_obl_ppm['Область ППМ'])


ostatok_with_block['Область ППМ'] = ostatok_with_block\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

spr_kod_okruga = rev_to_dict(spr_kod_okruga_['Область ППМ'].apply(lambda x: str(x).replace('.0',''))\
                             .astype('str'),spr_kod_okruga_['Код округа'])

spr_kod_okruga1 = rev_to_dict(T001W['Завод'].apply(lambda x: str(x).replace('.0','')).astype('str'),\
            T001W['Код округа'].apply(lambda x: str(x).replace('.0','')).astype('str'))

ostatok_with_block['округ'] = ostatok_with_block\
                                    .apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')


spr_pr_cs = rev_to_dict(spr_kod_okruga_ ['Область ППМ'].astype('str'),spr_kod_okruga_ ['Тип завода/области ППМ'].astype('str'))


ostatok_with_block['Признак ЦС'] = ostatok_with_block.apply(lambda x: insert_vpr(x['Область ППМ'],spr_pr_cs,'Тип завода/области ППМ') ,axis = 1).astype('str')


spr_pr_uchstiya_v_ppm =  rev_to_dict(T001L['Завод']+T001L['Склад'],T001L['Код ППМ'])

for st  in ['Завод','Склад','Принимающий склад']:
    prihod[st] = prihod[st].apply(lambda x: '' if x=='0' else x)

ostatok_with_block['Признак участия в ППМ'] = ostatok_with_block.apply(lambda x: insert_vpr(x['Завод']+x['Склад'],spr_pr_uchstiya_v_ppm,'Код ППМ') ,axis = 1).astype('str')




ostatok_with_block = ostatok_with_block[(ostatok_with_block['Склад']!='0')&\
                   (ostatok_with_block['Завод']!='1599')&\
                  (ostatok_with_block['Завод']!='1501')&\
                  (ostatok_with_block['Завод']!='1505')&\
                  (ostatok_with_block['Область ППМ']!='')&\
                   (ostatok_with_block['Признак участия в ППМ']!='1')&\
                   (ostatok_with_block['Признак ЦС']!='0')&\
                  (ostatok_with_block['Особый запас']!='Q')&\
                  ~((ostatok_with_block['Партия'].apply(lambda x:len(list(x))==2))&\
                    (ostatok_with_block['Партия']!=ostatok_with_block['округ']))]

ostatok_with_block = ostatok_with_block[ostatok_with_block['Материал'].apply(lambda x:x[:2]=='TT')|(ostatok_with_block['Материал'].apply(lambda x:  x[:2].isalpha()==False))]


# Убираем склады которые не уч в ППМ
aa = T001L[['Завод','Склад','Код ППМ']][T001L[['Завод','Склад','Код ППМ']]['Код ППМ']!='1']
ind = (ostatok_with_block['Завод']+ostatok_with_block['Склад']).isin(aa['Завод'].astype('str')+aa['Склад'].astype('str'))
ostatok_with_block = ostatok_with_block[ind]

ostatok_with_block = ostatok_with_block[ostatok_with_block['Особый запас']=='']

ostatok_with_block['цена на начало'] = ostatok_with_block.apply(lambda x: 0 if x['Кол-во запаса в начале периода']==0 else x['Стоимость в начале периода']/x['Кол-во запаса в начале периода'],axis = 1 )

ostatok_with_block['цена на конец'] = ostatok_with_block.apply(lambda x: 0 if x['Кол-во запаса в конце периода']==0 else x['Ст-ть запаса в конце периода']/x['Кол-во запаса в конце периода'] ,axis = 1 )


ostatok_with_block['Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса'] = ostatok_with_block.apply(lambda x: 0 if x['Блокированный запас']>x['Кол-во запаса в начале периода'] else x['Кол-во запаса в начале периода']-x['Блокированный запас'],axis = 1 )
ostatok_with_block['Стоимость в начале периода с учетом исключения блокированного запаси и особого запаса'] = ostatok_with_block['цена на начало']*ostatok_with_block['Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса']
ostatok_with_block['Кол-во запаса в конце периода с учетом исключения блокированного запаси и особого запаса'] = ostatok_with_block.apply(lambda x: 0 if x['Блокированный запас']>x['Кол-во запаса в конце периода'] else x['Кол-во запаса в конце периода']-x['Блокированный запас'],axis = 1 )
ostatok_with_block['Стоимость в конце периода с учетом исключения блокированного запаси и особого запаса'] = ostatok_with_block['цена на конец']*ostatok_with_block['Кол-во запаса в конце периода с учетом исключения блокированного запаси и особого запаса']


ostatok_with_block['измененная партия'] = ostatok_with_block.apply(lambda x:func1(x),axis = 1)




PICRL.fillna(0,inplace=True)
PICPS.fillna(0,inplace=True)
to_str_picps = ['PIC ключ','Материал']
for i in to_str_picps:
    PICPS[i] = PICPS[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    PICPS[i] = PICPS[i].apply(lambda x: '' if x=='0' else x)
PICPS['Материал'] = PICPS['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x) 
to_str_PICRL = ['PIC ключ','Ведущая деталь','Область ППМ','Завод']
for i in to_str_PICRL:
    PICRL[i] = PICRL[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    PICRL[i] = PICRL[i].apply(lambda x: '' if x=='0' else x)
    
PICRL['Ведущая деталь'] = PICRL['Ведущая деталь'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)   
spr_PICPS  = rev_to_dict(PICPS['Материал'],PICPS['PIC ключ'])
ostatok_with_block['PIC ключ']  = ostatok_with_block.apply(lambda x:'' if insert_vpr(x['Материал'],spr_PICPS,'PIC ключ')==0 else insert_vpr(x['Материал'],spr_PICPS,'PIC ключ'),axis = 1).astype('str')
PICRL['Область ППМ'] = PICRL['Область ППМ'].apply(lambda x:'' if x=='0' else x)
spr_PICRL_obl_ppm = rev_to_dict((PICRL['PIC ключ']+PICRL['Область ППМ']),PICRL['Ведущая деталь'])
spr_PICRL_obl_zav = rev_to_dict((PICRL['PIC ключ']+PICRL['Завод']),PICRL['Ведущая деталь'])
ostatok_with_block['Ведущий материал'] = ostatok_with_block.apply (lambda x:  insert_vpr((x['PIC ключ']+x['Завод']),spr_PICRL_obl_zav,'Ведущая деталь')\
                                                                   if insert_vpr((x['PIC ключ']+x['Область ППМ']),spr_PICRL_obl_ppm,'Ведущая деталь')==0\
                                                                   else  insert_vpr((x['PIC ключ']+x['Область ППМ']),spr_PICRL_obl_ppm,'Ведущая деталь') ,axis = 1)




to_str_ved_mat = ['Материал','Краткий текст материала','Базовая ЕИ','Область ППМ','Ведущий материал','Наименование ведущего материала']

for i in to_str_ved_mat:
    ved_mat[i] = ved_mat[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    ved_mat[i] = ved_mat[i].apply(lambda x: '' if x=='0' else x)
    
ved_mat['Материал'] = ved_mat['Материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x) 
ved_mat['Ведущий материал'] = ved_mat['Ведущий материал'].apply(lambda x: '0'+str(x) if  len(str(x))<9 else x)

ved_mat1 = rev_to_dict(ved_mat['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')\
            +ved_mat['Область ППМ'].apply(lambda x: str(x).replace('.0','')).astype('str'),ved_mat['Ведущий материал'].apply(lambda x: str(x).replace('.0','')).astype('str'))


ved_mat2 = rev_to_dict(ved_mat['Материал'].apply(lambda x: str(x).replace('.0','')).astype('str')\
            +ved_mat['Область ППМ'].apply(lambda x: str(x).replace('.0','')).astype('str'),ved_mat['Наименование ведущего материала'].apply(lambda x: str(x).replace('.0','')).astype('str'))

ostatok_with_block['материал с учетом аналогов'] = ostatok_with_block\
                .apply(lambda x: x['Материал'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал'),axis = 1).astype('str')


ostatok_with_block['наименование материал с учетом аналогов'] = ostatok_with_block\
                .apply(lambda x: x['КрТекстМатериала'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала'),axis = 1).astype('str')

ostatok_with_block = ostatok_with_block.groupby(['БЕ','материал с учетом аналогов','наименование материал с учетом аналогов','Базовая ЕИ','Завод','Область ППМ','округ','измененная партия','Ведущий материал'])\
                .agg({'Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса':'sum',\
                     'Стоимость в начале периода с учетом исключения блокированного запаси и особого запаса':'sum',\
                     'Кол-во запаса в конце периода с учетом исключения блокированного запаси и особого запаса':'sum',\
                     'Стоимость в конце периода с учетом исключения блокированного запаси и особого запаса':'sum','Кол-во запаса в конце периода':'sum'}).reset_index()

In [7]:
df['Дата потребности'] = df['Дата потребности'].apply(lambda x: '01.01.1900' if x =='00.00.0000' else  x)
df['Дата потребности'] = [datetime.datetime(int(x.split('.')[2]),int(x.split('.')[1]),int(x.split('.')[0])) for x in df['Дата потребности'].values]
df.fillna(0,inplace=True)
# собираю дату с группировкой по месяцу
month = df['Дата потребности'].apply(lambda x:  str(x.year)+'-'+str(x.month)+'-01'+ ' 00:00:00')
month =pd.to_datetime(month) 
# заменяю символ - 
df['Количество потребности'] = df['Количество потребности'].str.replace('-','')
df['Баланс (закрытие потребности)'] = df['Баланс (закрытие потребности)'].str.replace('-','')
# Работаю с числами
df['Вид документа потребности'] = df['Вид документа потребности'].astype('str')
df['Вид документа пополнения'] = df['Вид документа пополнения'].astype('str')
# меняем цифры на нормальные флоат
replace_float(df,['Количество потребности','Количество пополнения','Баланс (закрытие потребности)'])

df.fillna(0,inplace=True)

to_str_extr = ['Материал','Завод','Область ППМ','БЕ','Склад потребности','Особый запас','Клиент','Актуальный тип ППМ','АктСтатусМтрл на заводе','Заказ','Склад потребности','Инициатор']

for i in to_str_extr:
    df[i] = df[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    df[i] = df[i].apply(lambda x: '' if x=='0' else x)
    
    

df['Область ППМ'] = df\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад потребности'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')
df['Признак участия в ППМ'] = df.apply(lambda x: insert_vpr(x['Завод']+x['Склад потребности'],spr_pr_uchstiya_v_ppm,'Код ППМ') ,axis = 1) .astype('str')
df['округ'] = df.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')

df_total = df[(df['Склад потребности']!='')&\
    ((df['Область ППМ']!='')|(df['округ']!=''))&\
    (df['Особый запас']!='Q')&\
    (~df['Клиент'].apply(lambda x:x.startswith('I')))&\
    ((df['Актуальный тип ППМ']!='ND')|(df['АктСтатусМтрл на заводе']!='2'))]

df_total = df_total[df_total.apply(lambda x: False if  x['Актуальный тип ППМ']=='ND' or x['АктСтатусМтрл на заводе']=='2' else True ,axis = 1)]
df_total = df_total[df_total['Вид документа потребности'].isin(['UB02', 'AR', 'SB', 'Z1', 'Z3' ])]

df_total = df_total[df_total['Материал'].apply(lambda x:x[:2]=='TT')|(df_total['Материал'].apply(lambda x:  x[:2].isalpha()==False))]
df_total = df_total[df_total['Признак участия в ППМ']!='1']

# отсекает дату потребности по current date
df_total = df_total[df_total['Дата потребности'].apply(lambda x:x.month==current_date.month and x.year==current_date.year)]

df_total = df_total[df_total['Вид документа потребности'].apply(lambda x: x.find('ZUB')==-1)]

df_total['Модуль'] = df_total.apply(lambda x: modul(x),axis=1 )


df_total['материал с учетом аналогов'] = df_total\
                .apply(lambda x: x['Материал'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал'),axis = 1).astype('str')

df_total['наименование материал с учетом аналогов'] = df_total\
                .apply(lambda x: x['КрТекстМатериала'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала'),axis = 1).astype('str')

df__ = df_total.copy()
df_total = df_total.groupby(['БЕ','материал с учетом аналогов','наименование материал с учетом аналогов','Базовая ЕИ','Завод','Область ППМ','округ','Модуль','Склад потребности','Инициатор'])\
                .agg({'Количество потребности':'sum'}).reset_index()

In [8]:
prihod.fillna(0,inplace=True)
to_str_prihod = ['БЕ','Завод','Склад','Материал','Особый запас','Вид движения','Партия','Дата проводки','Краткий текст материала','Дата документа','Принимающая партия','Принимающий склад']
for i in to_str_prihod:
    prihod[i] = prihod[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    prihod[i] = prihod[i].apply(lambda x: '' if x=='0' else x)
    
    
spis_st_prih = ['БЕ','Завод','Склад','Материал','Особый запас','Вид движения','Партия','Дата проводки','Краткий текст материала','Дата документа','Принимающая партия','Принимающий склад']
prihod_ = prihod.groupby(spis_st_prih).agg({'Количество':'sum','Сумма во ВВ':'sum'}).reset_index()    

prihod = prihod_


prihod['Область ППМ'] = prihod\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

prihod['Признак участия в ППМ'] = prihod.apply(lambda x:pr_ppm(x,spr_pr_uchstiya_v_ppm)  ,axis = 1).astype('str')


prihod['округ'] = prihod.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') \
                        if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else \
                               insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')


prihod['Признак ЦС'] = prihod.apply(lambda x: \
                                insert_vpr(x['Область ППМ'],spr_pr_cs,'Тип завода/области ППМ') ,axis = 1).astype('str')

    




prihod = prihod[(prihod['Признак ЦС']=='1')&(prihod['Особый запас']=='')&(prihod['Завод']!='1599')&\
                ((prihod['Вид движения']=='101')|(prihod['Вид движения']=='102'))&\
       ((prihod['Партия']==prihod['округ'])|(prihod['Партия']=='DAMAGED')|(prihod['Партия']=='REPAIR')|\
       (prihod['Партия'].apply(lambda x: True if  x.isdigit() ==True and len(x)==10 else '')))]





ind = (prihod['Завод']+prihod['Склад']).isin(aa['Завод'].astype('str')+aa['Склад'].astype('str'))
prihod = prihod[ind]

prihod['Дата проводки'] = pd.to_datetime(prihod['Дата проводки'])
prihod['Декада проводки'] = prihod['Дата проводки'].apply(lambda x: '1 декада' if x.day>=1 and x.day<=10 else \
                                                                    '2 декада' if x.day>=11 and x.day<=20 else \
                                                                    '3 декада' if (x.day>=21 and x.day<=31) or (x.day>=21 and x.day<=30) else 0)


prihod['материал с учетом аналогов'] = prihod\
                .apply(lambda x: x['Материал'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал'),axis = 1)

prihod['наименование материал с учетом аналогов'] = prihod\
                .apply(lambda x: x['Краткий текст материала'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала'),axis = 1)



prihod['Дата документа'] =pd.to_datetime(prihod['Дата документа'])

# Расчет суммы в USD
TCURR_P = TCURR[(TCURR['Тип курса']=='M')& (TCURR['Исходная валюта']=='USD')]

spr_kurs = rev_to_dict(pd.concat([TCURR_P['Действит. с'].apply(lambda x:x),TCURR_P['Курс']],axis = 1)['Действит. с'],\
            pd.concat([TCURR_P['Действит. с'].apply(lambda x:x),TCURR_P['Курс']],axis = 1)['Курс'])

prihod['Сумма в ВВ в USD'] = prihod['Сумма во ВВ']/prihod.apply(lambda x:  insert_vpr(x['Дата документа'],spr_kurs,'Курс'),axis = 1)




prihod = prihod[(prihod['Количество']!=0)]



In [9]:
rashod.fillna(0,inplace=True)
to_str_rashod = ['БЕ','Завод','Склад','Материал','Особый запас','Вид движения','Заказ','Дата документа','Базовая ЕИ','Имя пользователя']
for i in to_str_rashod:
    rashod[i] = rashod[i].apply(lambda x: str(x).replace('.0','')).astype('str') 
    rashod[i] = rashod[i].apply(lambda x: '' if x=='0' else x)
rashod['Кол-во в ЕИ ввода']   =   rashod['Количество']

rashod.rename(columns={"Имя пользователя": "Инициатор"},inplace=True)


spis_st_rash = ['БЕ','Завод','Склад','Материал','Особый запас','Краткий текст материала','Вид движения','Заказ','Дата документа','Базовая ЕИ','Инициатор']
rashod_ = rashod.groupby(spis_st_rash).agg({'Кол-во в ЕИ ввода':'sum','Сумма во ВВ':'sum'}).reset_index()


rashod_['Кол-во в ЕИ ввода'] = rashod_['Кол-во в ЕИ ввода']*-1
rashod_['Сумма во ВВ'] = rashod_['Сумма во ВВ']*-1

rashod = rashod_
rashod['Область ППМ'] = rashod\
                                    .apply(lambda x: x['Завод'] if    insert_vpr(x['Завод']+x['Склад'],spr_obl_ppm,'Область ППМ')==0 else  \
                                     insert_vpr(x['Завод']+x['Склад'],spr_obl_ppm,'Область ППМ'),axis = 1).astype('str')

rashod['Признак участия в ППМ'] = rashod.apply(lambda x: insert_vpr(x['Завод']+x['Склад'],\
                                                                    spr_pr_uchstiya_v_ppm,'Код ППМ') ,axis = 1).astype('str')


rashod['округ'] = rashod.apply(lambda x: insert_vpr(x['Завод'],spr_kod_okruga1,'Код округа') \
                        if insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа')==0 else \
                               insert_vpr(x['Область ППМ'],spr_kod_okruga,'Код округа'),axis = 1).astype('str')


# фильтры
ind = (rashod['Завод']+rashod['Склад']).isin(aa['Завод'].astype('str')+aa['Склад'].astype('str'))
rashod = rashod[ind]

rashod = rashod[(rashod['Особый запас']!='Q')&\
       (rashod['Область ППМ']!='0')&(rashod['Признак участия в ППМ']!='1')&((rashod['округ']!='0')&(rashod['округ']!=''))]

ind_zak = rashod['Заказ'].apply(lambda x:False if x[:4]=='1017' else True)
rashod = rashod[ind_zak]

rashod = rashod[rashod['Материал'].apply(lambda x:x[:2]=='TT')|(rashod['Материал'].apply(lambda x:  x[:2].isalpha()==False))]

rashod['материал с учетом аналогов'] = rashod\
                .apply(lambda x: x['Материал'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat1,'Ведущий материал'),axis = 1)

rashod['наименование материал с учетом аналогов'] = rashod\
                .apply(lambda x: x['Краткий текст материала'] if    insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала')==0 else  \
                insert_vpr(x['Материал']+x['Область ППМ'],ved_mat2,'Наименование ведущего материала'),axis = 1)

rashod['Модуль'] = rashod.apply(lambda x: modul1(x),axis=1 )

rashod['Дата документа'] =pd.to_datetime(rashod['Дата документа'])



def rep(df,cols):
    for i in cols:
        df[i] = df[i].apply(lambda x: str(x).replace('.0','')).astype('str')
        df[i] = df[i].apply(lambda x: '' if x=='0' or x==0 or x =='nan' or x=='#' else x)
    return df  


zap_col_st.columns = [x[1]+str(zap_col_st.loc[0].values[x[0]] )if x[1].find('Unnamed')==-1 else zap_col_st.loc[0].values[x[0]]  for x in enumerate(zap_col_st.columns) ]
zap_col_st = header_parser(zap_col_st)
zap_col_st = zap_col_st[zap_col_st.index!=0].reset_index(drop=True)
# rashod_start.fillna(0,inplace=True)
sp_rash = ['Материал','Завод','Склад','Вид движения','БЕ','Особый запас']
rashod = rep(rashod,sp_rash)
zap_col_st.fillna(0,inplace=True)
zap_col_st['Цена'] = zap_col_st.apply(lambda x: x['Стоимость отпущенного\nзапаса (USD)nan']/x['Кол-во отпуска\nзапасаnan'] if x['Кол-во отпуска\nзапасаnan']!=0 else 0,axis = 1)
rashod = rep(rashod,['БЕ','Завод','Склад','Вид движения','Материал','Особый запас'])
zap_col_st['Тип запасов'].replace('#','',inplace = True)
zap_col_st = rep(zap_col_st,['Балансовая единица','Завод','Склад','Вид потока','Материал','Тип запасов'])

zap_col_st.rename(columns={"Балансовая единица": "БЕ",'Вид потока':'Вид движения','Тип запасов':'Особый запас'},inplace=True)
gr_zap_col_ = zap_col_st[['БЕ','Завод','Склад','Вид движения','Материал','Цена','Особый запас']].drop_duplicates()
rrrr =  pd.merge(rashod,gr_zap_col_.groupby(['БЕ','Завод','Склад','Вид движения','Материал','Особый запас']).mean().reset_index(),how='left',left_on=['БЕ','Завод','Склад','Вид движения','Материал','Особый запас'],right_on=['БЕ','Завод','Склад','Вид движения','Материал','Особый запас'])

print('Проверка цена_остаток '+str(rrrr['Сумма во ВВ'].sum()-rashod['Сумма во ВВ'].sum()))


rashod =rrrr.copy() 


rashod['Сумма в ВВ в USD'] = rashod['Цена']*rashod['Кол-во в ЕИ ввода']

rashod = rashod.groupby(['Базовая ЕИ','БЕ','материал с учетом аналогов','наименование материал с учетом аналогов','Завод','Область ППМ','округ','Модуль','Склад','Инициатор'])\
                .agg({'Кол-во в ЕИ ввода':'sum','Сумма во ВВ':'sum','Сумма в ВВ в USD':'sum'}).reset_index()


rashod = rashod[(rashod['Кол-во в ЕИ ввода']>0)&(rashod['Сумма во ВВ']>=0)]

Проверка цена_остаток 0.0


In [10]:

# df_total.drop('наименование материал с учетом аналогов',axis = 1,inplace = True)
df_total.rename(columns={"Склад потребности": 'Склад_'},inplace=True)
rashod.rename(columns={"Склад": 'Склад_'},inplace=True)
df_total['Склад_'] = df_total['Склад_'].apply(lambda x: '0'+str(x) if  len(str(x))<4 else x)
rashod['Склад_'] = rashod['Склад_'].apply(lambda x: '0'+str(x) if  len(str(x))<4 else x)
cons = pd.merge(left=df_total,right=rashod[['материал с учетом аналогов','Область ППМ','Модуль','Кол-во в ЕИ ввода','БЕ','Базовая ЕИ','Завод','округ','Сумма во ВВ','Склад_','Сумма в ВВ в USD','Инициатор']],\
         on=['материал с учетом аналогов','Область ППМ','Модуль','БЕ','Базовая ЕИ','Завод','округ','Склад_','Инициатор'],how='outer')
cons.fillna(0,inplace = True)
cons.reset_index(drop = True,inplace = True)
cons.rename(columns={"материал с учетом аналогов": 'Материал'},inplace=True)

In [11]:
# Плановая цена
KONH.fillna(0,inplace=True)
KONP.fillna(0,inplace=True)
MARM.fillna(0,inplace=True)

to_str_KONH = ['Номер записи условий','Таблица','Переменный ключ','Вид условия']
for i in to_str_KONH:
    KONH[i] = KONH[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    KONH[i] = KONH[i].apply(lambda x: '' if x=='0' else x)
    
to_str_KONP = ['Номер записи условий','Единица измерения','Вид условия','Валюта условия']
for i in to_str_KONP:
    KONP[i] = KONP[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    KONP[i] = KONP[i].apply(lambda x: '' if x=='0' else x)
KONP = KONP[KONP['Индикатор удаления']!='X']    
KONH = KONH[(KONH['Вид условия']=='ZP00')&((KONH['Таблица']=='900')|(KONH['Таблица']=='901')|(KONH['Таблица']=='902'))]   
KONH['Материал'] = KONH.apply(lambda x:func_KONH(x),axis = 1)

KONH['округ'] = KONH.apply(lambda x: x['Переменный ключ'][:2] if x['Таблица']=='902' else 'пусто',axis = 1)
KONH['Завод'] = KONH.apply(lambda x: x['Переменный ключ'][:4] if x['Таблица']=='901' else 'пусто',axis = 1)
KONH.reset_index(drop = True,inplace = True)


to_str_MARM = ['Материал','Альтернативная ЕИ']
for i in to_str_MARM:
    MARM[i] = MARM[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    MARM[i] = MARM[i].apply(lambda x: '' if x=='0' else x)


for d in [A900,A901,A902]:
    d['Номер записи условий'] =  d['Номер записи условий'].apply(lambda x: str(x).replace('.0','')).astype('str')
    
fil = KONH['Номер записи условий'].isin(pd.concat([A900['Номер записи условий'],A901['Номер записи условий'],A902['Номер записи условий']]).drop_duplicates())
KONH = KONH[fil]  

cons = parallel_calc_fast(cons,KONH,current_date,KONP,MARM)

--- 326.7868082523346 seconds ---


In [12]:

cons['Признак ЦС'] = cons.apply(lambda x: insert_vpr(x['Область ППМ'],spr_pr_cs,'Тип завода/области ППМ') ,axis = 1).astype('str')


spr_MDLV = rev_to_dict(MDLV['Область ППМ'],MDLV['Текст ОблПланирПотр']) 
cons['Склад'] = cons.apply(lambda x:insert_vpr(x['Область ППМ'],spr_MDLV,'Текст ОблПланирПотр'),axis = 1).astype('str')

spr_MDLV_sk_post = rev_to_dict(MDLV['Область ППМ'],MDLV['СкладПоступления']) 
cons['Склад поступления'] = cons.apply(lambda x: insert_vpr(x['Область ППМ'],spr_MDLV_sk_post,'СкладПоступления') if x['Признак ЦС']=='3' else 0  ,axis = 1).astype('str')



to_str_ZMME0078 = ['Область ППМ','Код округа','Вид ОблПланирПотребн','Партия','Тип завода/области ППМ']

for i in to_str_ZMME0078:
    spr_kod_okruga_[i] = spr_kod_okruga_[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    spr_kod_okruga_[i] = spr_kod_okruga_[i].apply(lambda x: '' if x=='0' else x)
    

    
to_str_T001W = ['Завод','Код округа','Закуп. организация']

for i in to_str_T001W:
    T001W[i] = T001W[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    T001W[i] = T001W[i].apply(lambda x: '' if x=='0' else x)


spr2 = rev_to_dict((T001W['Завод']+T001W['Код округа']),T001W['Закуп. организация'])
spr_kod_okruga_2 = spr_kod_okruga_[spr_kod_okruga_['Тип завода/области ППМ']=='1']
spr_kod_okruga_2['Завод'] = spr_kod_okruga_2['Область ППМ'].apply(lambda x:str(x)[:4])
spr_kod_okruga_2['БЕ'] = spr_kod_okruga_2.apply(lambda x:'' if insert_vpr(x['Завод']+x['Код округа'],spr2,'Закуп. организация')==0\
                                                else insert_vpr(x['Завод']+x['Код округа'],spr2,'Закуп. организация') ,axis = 1)
aa = spr_kod_okruga_2[['БЕ','Код округа','Область ППМ','Завод']]
spr3 = rev_to_dict((aa['БЕ'].astype('str')+aa['Код округа'].astype('str')),aa['Завод'])
cons['Завод пополнения'] = cons.apply(lambda x:x['Завод'] if insert_vpr(x['БЕ']+x['округ'],spr3,'Завод')==0 else\
                                      insert_vpr(x['БЕ']+x['округ'],spr3,'Завод'),axis = 1).astype('str')



cons['сортировка по модулю'] = cons.apply(lambda x: sort_for_modul(x),axis=1 )#по ней будем сортировтаь распределение поэтому в строку не превращаем

cons['партия МВП'] = cons.apply(lambda x: part_MVP(x),axis=1 ).astype('str')

cons['партия на заводе ЦС'] = cons.apply(lambda x: part_na_zav_CS(x),axis=1 ).astype('str')

cons = cons[cons['Модуль']!='']
cons = cons[cons['Материал'].apply(lambda x:len(x)<=10)]
cons.reset_index(drop=True,inplace = True)
cons.fillna(0,inplace=True)

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# мехника распределения остатков 
# col_name_0 название столбца откуда рпспределяем
# col_name_1 навание с толбца куда распределяем
# col_name_2 название столбца с количеством потребности
# col_name_5 название столбца куда распределяем сумму
# Для цены:
# col_name_3 название столбца с суммой  в остатках
# col_name_4 название столбца с начальным кол-вом  потребности
start_time = time.time()
ostatok_with_block.rename(columns={"материал с учетом аналогов": 'Материал','измененная партия':'Партия'},inplace=True)
potr_copy = cons.copy()
ost_copy = ostatok_with_block.copy()
# Запуск распределения остатков
potr_copy['Распределение_остатака  на начало'] = 0.000000
potr_copy['Сумма Распределение_остатака  на начало'] = 0.000000
potr_copy['index'] = [x for x in range(len(potr_copy))]
ost_copy['осталось'] = ost_copy['Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса']

ostat(ost_st=ost_copy,potr_st=potr_copy,col_name_0='осталось',col_name_1='Распределение_остатака  на начало',col_name_2 = 'Количество потребности',col_name_3 = 'Стоимость в начале периода с учетом исключения блокированного запаси и особого запаса',col_name_4 = 'Кол-во запаса в начале периода с учетом исключения блокированного запаси и особого запаса',col_name_5 = 'Сумма Распределение_остатака  на начало')
# откуда куда что
print("--- %s seconds ---" % (time.time() - start_time))


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.1 закончили


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.2 закончили


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.3 закончили
--- 182.09293842315674 seconds ---


In [14]:
start_time = time.time()
potr_copy['Количество для распределения на конец периода'] = \
potr_copy.apply(lambda x: 0 if (x['Количество потребности']-x['Кол-во в ЕИ ввода'])<0 \
                else x['Количество потребности']-x['Кол-во в ЕИ ввода'],axis = 1 )

potr_copy1 = potr_copy.copy()
ost_copy1 = ost_copy.copy()
# Запуск распределения остатков
potr_copy1['Распределение_остатака  на конец'] = 0.000000
potr_copy1['Сумма Распределение_остатака  на конец'] = 0.000000
ost_copy1['осталось1'] = ost_copy1['Кол-во запаса в конце периода с учетом исключения блокированного запаси и особого запаса']

ostat(ost_st=ost_copy1,potr_st=potr_copy1,col_name_0='осталось1',col_name_1='Распределение_остатака  на конец',col_name_2 = 'Количество для распределения на конец периода',col_name_3 = 'Стоимость в конце периода с учетом исключения блокированного запаси и особого запаса',col_name_4 = 'Кол-во запаса в конце периода с учетом исключения блокированного запаси и особого запаса',col_name_5 = 'Сумма Распределение_остатака  на конец')
print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.1 закончили


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.2 закончили


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.3 закончили
--- 173.84882736206055 seconds ---


In [15]:

start_time = time.time()
potr_copy1['Количество для распределения прихода'] = potr_copy1.apply(lambda x: x['Количество потребности']-x['Распределение_остатака  на начало']  \
                                                                               if x['Распределение_остатака  на начало']-x['Количество потребности']<0 else 0,axis = 1)


potr_copy2 = potr_copy1.copy()
prihod_copy = prihod.copy()
prihod_copy = prihod_copy[prihod_copy['Количество']>0]
potr_copy2['Распределение_прихода'] = 0.000000

potr_copy2['1 декада'] = 0.000000
potr_copy2['2 декада'] = 0.000000
potr_copy2['3 декада'] = 0.000000


prihod_copy['осталось'] = prihod_copy['Количество']



prihod_raspr(ost_st=prihod_copy,potr_st=potr_copy2,col_name_0='осталось',col_name_1='Распределение_прихода',col_name_2 = 'Количество для распределения прихода')
print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.1 закончили


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.2 закончили


C:\Users\NKedrun\Desktop\Portable Python 3.6.5 x64\App\Python\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


15.2.3 закончили
--- 40.6094446182251 seconds ---


In [16]:
from decimal import Decimal
for col in [x for x in potr_copy2.columns if potr_copy2[x].dtype=='float64']:
    potr_copy2[col] = potr_copy2[col].apply(lambda x: round( Decimal(x),3))
    
potr_copy2['Погрешность прихода'] = potr_copy2.apply(lambda x: x['Кол-во в ЕИ ввода']-(x['Распределение_остатака  на начало']+x['1 декада']+x['2 декада']) \
                 if x['Кол-во в ЕИ ввода']>(x['Распределение_остатака  на начало']+x['1 декада']+x['2 декада']) else 0,axis = 1)
potr_copy2['3 декада'] = potr_copy2.apply(lambda x: 0 if x['Распределение_прихода']<=(x['1 декада']+x['2 декада']+x['Погрешность прихода']) else \
                 x['Распределение_прихода']-x['1 декада']-x['2 декада']-x['Погрешность прихода'],axis = 1)

potr_copy2['Отклонение ФС-ПП кол-во'] = potr_copy2['Кол-во в ЕИ ввода']-potr_copy2['Количество потребности']
potr_copy2['Отклонение ФС-ПП %'] = potr_copy2.apply(lambda x:proc_otklon(x),axis = 1)

potr_copy2['Обеспеченность ПП запасом на начало периода кол-во'] = potr_copy2.apply(lambda x: x['Распределение_остатака  на начало']  if x['Распределение_остатака  на начало']<\
                                                                                    x['Количество потребности'] else x['Количество потребности'] ,axis = 1)

potr_copy2['Обеспеченность ПП запасом на начало периода %'] = potr_copy2.apply(lambda x: 1 if x['Количество потребности'] ==0 else x['Обеспеченность ПП запасом на начало периода кол-во']/\
                 x['Количество потребности'],axis = 1)



potr_copy2['Обеспеченность ПП запасом на начало периода и ФП кол-во'] = potr_copy2.apply(lambda x: x['Распределение_остатака  на начало']+x['1 декада']+x['3 декада']+x['2 декада']+x['Погрешность прихода'] \
                 if (x['Распределение_остатака  на начало']+x['1 декада']+x['3 декада']+x['2 декада']+x['Погрешность прихода'])<x['Количество потребности'] else\
                 x['Количество потребности'],axis = 1)

potr_copy2['Обеспеченность ПП запасом на начало периода и ФП %'] = potr_copy2.apply(lambda x: 1 if x['Количество потребности'] ==0 else\
                 x['Обеспеченность ПП запасом на начало периода и ФП кол-во']/x['Количество потребности'],axis = 1)





potr_copy2['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) кол-во'] = potr_copy2.apply(lambda x: x['Распределение_остатака  на начало']+x['1 декада']+x['2 декада']+x['Погрешность прихода'] \
                 if (x['Распределение_остатака  на начало']+x['1 декада']+x['2 декада']+x['Погрешность прихода'])<x['Количество потребности'] else\
                 x['Количество потребности'],axis = 1)






potr_copy2['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %'] = potr_copy2.apply(lambda x: 1 if x['Количество потребности'] ==0 else\
                 x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) кол-во']/x['Количество потребности'],axis = 1)






potr_copy2['Примечание по списанию'] = potr_copy2.apply(lambda x:primechaniya_po_spis(x),axis = 1)
potr_copy2['Примечание по обеспеченности'] = potr_copy2.apply(lambda x:primechaniya_po_obespech(x),axis = 1)



    
potr_copy2['KPI по вовлечению'] = potr_copy2['Отклонение ФС-ПП %']

for col in [x for x in potr_copy2.columns if potr_copy2[x].dtype=='float64']:
    potr_copy2[col] = potr_copy2[col].apply(lambda x: round( Decimal(x),3))
    
potr_copy2['KPI по причинам'] = potr_copy2.apply(lambda x:KPI_prichinam(x),axis = 1)
potr_copy2['красная'] = 0.8
potr_copy2['желтая'] = 0.15
potr_copy2['зеленая'] = 0.05

to_str_T023 = ['Группа материалов','Обозначение 2 группы материалов']

for i in to_str_T023:
    T023[i] = T023[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    T023[i] = T023[i].apply(lambda x: '' if x=='0' else x)
    
spr_group_mat_l1 = rev_to_dict(T023[T023['Группа материалов'].str.len()==2]['Группа материалов'],T023[T023['Группа материалов'].str.len()==2]['Обозначение 2 группы материалов'])
potr_copy2['группа'] = potr_copy2['Материал'].str[:2].apply(lambda x: str(x) +' '+ insert_vpr(x,spr_group_mat_l1,'Обозначение 2 группы материалов'))

to_str_spr_kompleks = ['Завод','Склад']

for i in to_str_spr_kompleks:
    spr_kompleks[i] = spr_kompleks[i].apply(lambda x: str(x).replace('.0','')).astype('str')
    spr_kompleks[i] = spr_kompleks[i].apply(lambda x: '' if x=='0' else x)
    
spr_zav_kompl = rev_to_dict(spr_kompleks['Завод']+spr_kompleks['Склад'],spr_kompleks['Критерий поиска 1'])    
potr_copy2['Комплекс'] = (potr_copy2['Завод']+potr_copy2['Склад_']).apply(lambda x:insert_vpr(x,spr_zav_kompl,'Критерий поиска 1'))

potr_copy2['Отклонение по KPI вовлечения'] = 1-potr_copy2['KPI по вовлечению']

potr_copy2['Условное отклонение ФС-ПП, сумма USD'] = potr_copy2.apply(lambda x: abs(x['Отклонение ФС-ПП кол-во']*x['Плановая цена'])\
                 if x['Отклонение ФС-ПП кол-во']<0 else 0,axis = 1)
potr_copy2['Отклонение  заказчика, сумма USD'] = potr_copy2.apply(lambda x:(x['Количество потребности']-x['Кол-во в ЕИ ввода'])*x['Плановая цена'] \
                 if (x['Количество потребности']>x['Кол-во в ЕИ ввода']) and (x['Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %']==Decimal('1')) else 0,axis = 1)
potr_copy2['Отклонение закупщика, сумма USD'] = potr_copy2.apply(lambda x:otklon_zak_sum_USD(x),axis = 1)

# potr_copy2['Прочие отклонения, сумма USD'] = potr_copy2['Условное отклонение ФС-ПП, сумма USD']-(potr_copy2['Отклонение  заказчика, сумма USD']+potr_copy2['Отклонение закупщика, сумма USD'])

potr_copy2['Сумма потребности, USD']  = potr_copy2['Количество потребности']*potr_copy2['Плановая цена']

potr_copy2['Сумма потребности, тенге'] = potr_copy2['Сумма потребности, USD']*375

potr_copy2['Показатель по вовлечению обеспеченной потребности'] = potr_copy2.apply(lambda x:pokaz_vovl(x),axis = 1)

potr_copy2.drop(['номер записи условий','цена без учета коэф.пересчата','единица измерения закупки','Признак ЦС',
                'Склад поступления','Завод пополнения','сортировка по модулю','партия МВП','партия на заводе ЦС','index',
                'Количество для распределения на конец периода','Количество для распределения прихода'],axis = 1,inplace = True)

In [17]:
potr_copy2['наименование материала']=''

In [18]:
potr_copy2.rename(columns={"Обеспеченность ПП запасом на начало периода и ФП кол-во": 'Обеспеченность ПП запасом на начало периода, ФП  и погрешностью прихода, кол-во'\
                          ,'Обеспеченность ПП запасом на начало периода и ФП %':'Обеспеченность ПП запасом на начало периода, ФП  и погрешностью прихода %'\
                          ,'Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) кол-во':'Обеспеченность ПП запасом на начало периода, ФП (1 и 2 декада) и погрешностью прихода кол-во'\
                          ,'Обеспеченность ПП запасом на начало периода и ФП (1 и 2 декада) %':'Обеспеченность ПП запасом на начало периода, ФП (1 и 2 декада) и погрешностью прихода %'\
                          ,'Кол-во в ЕИ ввода':'Количество по списанию','Сумма во ВВ':'Сумма по списанию','Сумма в ВВ в USD':'Сумма в списанию в USD'},inplace=True)

ex = pd.read_excel('C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\Копия KPI_.xlsx',sheet_name='sheet1')
potr_copy2  = potr_copy2[list(ex.T.sort_index()[0].values)]


df_name = rev_to_dict(df__['материал с учетом аналогов'],df__['наименование материал с учетом аналогов'])
rashod_name = rev_to_dict(rashod['материал с учетом аналогов'],rashod['наименование материал с учетом аналогов'])




potr_copy2['наименование материала'] = potr_copy2\
                .apply(lambda x: '' if    insert_vpr(x['Материал'],df_name,'наименование материал с учетом аналогов')==0 else  \
                insert_vpr(x['Материал'],df_name,'наименование материал с учетом аналогов'),axis = 1).astype('str')

potr_copy2['наименование материала'] = potr_copy2\
                .apply(lambda x:  insert_vpr(x['Материал'],rashod_name,'наименование материал с учетом аналогов') if x['наименование материала']=='' else  \
                x['наименование материала'],axis = 1).astype('str')

In [19]:
potr_copy2.rename(columns={"Сумма в списанию в USD": 'Сумма по списанию в USD'},inplace=True)
potr_copy2.columns = [x.replace('остатака','остатка') if x.find('остатака')!=-1 else x for x in  potr_copy2.columns]

In [ ]:
save_pyex(df = ostatok_with_block ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\ostat'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
save_pyex(df = df_total ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\PP'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
save_pyex(df = prihod ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\prihod'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
save_pyex(df = rashod ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\rashod'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
save_pyex(df = cons ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\cons'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
save_pyex(df = potr_copy2 ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\KPI'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
x.replace('остатака','остатка') x.find('остатака')!=-1 else x

In [ ]:
df = pd.read_csv(directory+'CSV.csv',sep = ';',parse_dates=['Дата потребности'],dtype={'Материал':'str','Завод':'str'})

In [ ]:
df

In [ ]:
'Распределение_остатака  на начало'.replace('остатака','остатка')

In [ ]:
df

In [ ]:
rashod

In [ ]:
save_pyex(df = df_total ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\df!!!!!'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
potr_copy2

In [21]:
save_pyex(df = potr_copy2 ,path='C:\\Users\\NKedrun\\Desktop\\kazzinc\\KPI\\files\\KPI_with_initial'+str(current_date.month)+'_'+str(current_date.year)+'.xlsx')

In [ ]:
df[['Резервирование']]

In [ ]:
df[['Номер потребности','Позиция элемента ППМ']]

In [ ]:
rashod[['Резервирование','№Поз РезервированПеремещЗапаса']]

In [ ]:
df